In [1]:
import os
from dataclasses import dataclass
import torch
import numpy as np
import math
from tqdm import tqdm
import json 
import pandas as pd

from src.tokenizers.bpe.eng import MyBPETokenizer
from src.tokenizers.bpe.indic import IndicBPETokenizer
from src.components import TransformerMT as selftx
from src.torchlayers import TransformerMT as torchtx

In [2]:
def translate_batch(model, eng_tokenizer, indic_tokenizer, english_texts, device='cuda', verbose=False):

    # all texts to sequences at once
    english_ids = eng_tokenizer.texts_to_sequences(english_texts)
    english_tensor = torch.tensor(english_ids, device=device)
    
    with torch.no_grad():
        # generate translations for the entire batch from model.generate
        translation_ids = model.generate(english_tensor, max_length=20, temperature=0.0)
    
    if verbose:
        print(f"Batch size: {len(english_texts)}")
        print(f"Generated shape: {translation_ids.shape}")

    # decode from indic_detokenizer
    translation_array = translation_ids.cpu().numpy()
    indic_texts = indic_tokenizer.sequences_to_texts(translation_array)
    
    return indic_texts

In [3]:
test_data = json.load(open(os.path.join("data", "raw", "val_data1.json")))

### ENG 2 HINDI

In [9]:
sorted(os.listdir("checkpoints/eng_hindi"))

['exp3-eng-hindi-transformer',
 'exp4-eng-hindi-transformer-built-in',
 'exp5-eng-hindi-transformer-built-in-bpe']

In [5]:
@dataclass
class TransformerConfig:
    SRC_VOCAB_SIZE: int = 20_000                      # source vocabulary size
    TGT_VOCAB_SIZE: int = 20_000                      # target vocabulary size
    SRC_MAX_LENGTH: int = 256                         # max sequence length source lang
    TGT_MAX_LENGTH: int = 256                         # max sequence length target lang
    D_MODEL: int = 128                                # embedding dimension
    N_HEADS: int = 4                                  # number of heads in attention
    N_LAYERS: int = 6                                 # number of transformer blocks
    D_FF: int = 128 * 4                               # dimension of feedforward (4x of embedding dims)
    MAX_SEQ_LEN: int = 256
    DROPOUT: float = 0.1
    BATCH_SIZE: int = 32
    EVAL_STEPS: int = 250
    EPOCHS: int = 10

In [13]:
config = TransformerConfig()

In [14]:
hindi_checkpoint_path = os.path.join("checkpoints", "eng_hindi", "exp5-eng-hindi-transformer-built-in-bpe")
checkpoint = torch.load(os.path.join(hindi_checkpoint_path, "tx_epoch_10_step_22500.pt"), weights_only=False)

In [15]:
# eng tokenizer
eng_tokenizer = MyBPETokenizer(config.SRC_VOCAB_SIZE)
eng_tokenizer = eng_tokenizer.load_tokenizer("tokenizers/english_shared_bpe_20000_256.pkl")

loaded tokenizer from tokenizers/english_shared_bpe_20000_256.pkl


In [18]:
# indic tokenizer
hi_tokenizer = IndicBPETokenizer()
hi_tokenizer= hi_tokenizer.load_tokenizer("tokenizers/hindi_bpe_20000_256.pkl")

loaded tokenizer from tokenizers/hindi_bpe_20000_256.pkl


In [16]:
model_config = checkpoint['model_config']
model = torchtx(**model_config)
model.to('cuda')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [19]:
translation = translate_batch(model, eng_tokenizer, hi_tokenizer, english_texts=["Hi"] , device='cuda')

converting texts: 100%|████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11491.24it/s]
/home/decision/anaconda3/envs/nlp/lib/python3.12/site-packages/torch/nn/modules/transformer.py:515: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(


In [22]:
hindi_output = pd.DataFrame()
ids = []
texts = []
for id_, entry in test_data['English-Hindi']['Validation'].items():
    ids.append(id_)
    texts.append(entry['source'])

print(f"Total samples: {len(texts)}")
print(f"Processing in batches of {32}")

all_translations = []
for i in tqdm(range(0, len(texts), 32), desc="Processing batches"):
    batch_texts = texts[i:i+32]
    batch_translations = translate_batch(model, eng_tokenizer, hi_tokenizer, batch_texts, device='cuda')
    all_translations.extend(batch_translations)

# gather in df
hindi_output['ID'] = ids
hindi_output['Translation'] = all_translations

Total samples: 11543
Processing in batches of 32


converting texts: 100%|██████████████████████████████████████████████████████| 1/1 [00:00<00:00, 154.87it/s]


converting texts: 100%|██████████████████████████████████████████████████████| 1/1 [00:00<00:00, 155.26it/s]


converting texts: 100%|██████████████████████████████████████████████████████| 1/1 [00:00<00:00, 135.16it/s]


converting texts: 100%|██████████████████████████████████████████████████████| 1/1 [00:00<00:00, 194.51it/s]


...

converting texts: 100%|██████████████████████████████████████████████████████| 1/1 [00:00<00:00, 139.60it/s]


converting texts: 100%|███████████████████████████████████████████████████████| 1/1 [00:00<00:00, 88.65it/s]


converting texts: 100%|██████████████████████████████████████████████████████| 1/1 [00:00<00:00, 113.72it/s]


converting texts: 100%|██████████████████████████████████████████████████████| 1/1 [00:00<00:00, 110.32it/s]


Processing batches: 100%|█████████████████████████████████████████████████| 361/361 [01:26<00:00,  4.19it/s]


In [63]:
ix = np.random.choice(list(test_data['English-Hindi']['Validation'].keys()))
print(ix)
print(test_data['English-Hindi']['Validation'].get(ix))
print(hindi_output.loc[hindi_output['ID'] == ix]['Translation'].values[0])

506047
{'source': 'In response to a question , she said that beauty is in the eyes of the onlookers .'}
यह प्रक्रिया उस बात पर कहा जाता है कि वह


### ENG 2 BENGALI

In [65]:
@dataclass
class TransformerConfig:
    SRC_VOCAB_SIZE: int = 20_000                      # source vocabulary size
    TGT_VOCAB_SIZE: int = 20_000                      # target vocabulary size
    SRC_MAX_LENGTH: int = 256                         # max sequence length source lang
    TGT_MAX_LENGTH: int = 256                         # max sequence length target lang
    D_MODEL: int = 256                                # embedding dimension
    N_HEADS: int = 4                                  # number of heads in attention
    N_LAYERS: int = 6                                 # number of transforme~r blocks
    D_FF: int = 256 * 4                               # dimension of feedforward (4x of embedding dims)
    MAX_SEQ_LEN: int = 256
    DROPOUT: float = 0.1
    BATCH_SIZE: int = 32
    EVAL_STEPS: int = 250
    EPOCHS: int = 10

In [66]:
bengali_checkpoint_path = os.path.join("checkpoints", "eng_bengali", "exp5-eng-bengali-transformer-built-in-bpe")
checkpoint = torch.load(os.path.join(bengali_checkpoint_path, "tx_epoch_10_step_19000.pt"), weights_only=False)

In [67]:
# indic tokenizer
be_tokenizer = IndicBPETokenizer()
be_tokenizer= hi_tokenizer.load_tokenizer("tokenizers/bengali_bpe_20000_256.pkl")

loaded tokenizer from tokenizers/bengali_bpe_20000_256.pkl


In [68]:
model_config = checkpoint['model_config']
model = torchtx(**model_config)
model.to('cuda')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [70]:
translation = translate_batch(model, eng_tokenizer, be_tokenizer, english_texts=["Hi"] , device='cuda')

converting texts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 7943.76it/s]


In [72]:
bengali_output = pd.DataFrame()
ids = []
texts = []
for id_, entry in test_data['English-Bengali']['Validation'].items():
    ids.append(id_)
    texts.append(entry['source'])

print(f"Total samples: {len(texts)}")
print(f"Processing in batches of {32}")

all_translations = []
for i in tqdm(range(0, len(texts), 32), desc="Processing batches"):
    batch_texts = texts[i:i+32]
    batch_translations = translate_batch(model, eng_tokenizer, be_tokenizer, batch_texts, device='cuda')
    all_translations.extend(batch_translations)

bengali_output['ID'] = ids
bengali_output['Translation'] = all_translations

Total samples: 9836
Processing in batches of 32


converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 242.36it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 239.20it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 209.91it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 281.14it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 202.55it/s]

converting texts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 171.82it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 172.01it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 176.94it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 194.18it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 197.98it/s]

converting texts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 123.27it/s]


converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 134.17it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 140.36it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 154.36it/s]

Processing batches:  35%|███████████████████████████████████████▎                                                                         | 107/308 [00:10<00:20,  9.66it/s]


...

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 129.03it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 131.99it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 139.33it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 157.14it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 138.00it/s]

converting texts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 131.16it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 134.14it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 230.77it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 218.66it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 211.16it/s]

converting texts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 218.78it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 208.71it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 188.11it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 148.00it/s]

converting texts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 132.76it/s]

converting texts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████

Processing batches:  98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▍  | 301/308 [00:29<00:00,  9.22it/s]


Processing batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 308/308 [00:30<00:00, 10.14it/s]


## Final output for submission

In [73]:
import csv

In [74]:
output = pd.concat([bengali_output, hindi_output]).reset_index(drop=True)

In [75]:
# output['Translation'] = output['Translation'].str.replace(",", "")

In [76]:
output.to_csv(
    "answers/val/answer5.csv",
    sep="\t",
    index=False,
    header=True,
    quoting=csv.QUOTE_ALL,
    lineterminator="\n",
    doublequote=True,
)

In [36]:
# output.to_csv("answers/val/answer1.csv", index=False)

In [93]:
# answer = "answers/val/answer1.csv"
# with open(answer, "w", newline="", encoding="utf-8") as f:
#     writer = csv.writer(f, delimiter="\t", quoting=csv.QUOTE_MINIMAL)
#     writer.writerow(["ID", "Translation"])  # header
#     for i in range(output.shape[0]):
#         writer.writerow([output["ID"][i], output["Translation"][i]])